# Sentiment Analyzer Notebook

This script imports a collection of Tweets by Elon Musk into a Pandas dataframe. It then processes the text of those Tweets through the Meaningcloud Sentiment Analyzer API. The response data from the API is then saved back into the dataframe and exported as a CSV.

Respones from Meaningcloud for Sentiment:

P+   = Strong Positive
P    = Positive
NEU  = Neutral
N    = Negative
N+.  = Strong Negative
NONE = No Polarity

Respone from Meaningcloud API for Confidence:

A number from 0-100, with 100 being the most confidence in the Sentiment score returned.


In [ ]:
# Import required libraries
import re
import requests
import pandas as pd
from textblob import TextBlob
import os
from dotenv import load_dotenv
import time
from pathlib import Path

load_dotenv()

In [ ]:
# Import Tweets from Kaggle CSV into a dataframe
sentiment_df = pd.read_csv('KaggleData/TweetsElonMusk.csv')

# Preview the dataframe
sentiment_df.head()

In [ ]:
# Create a column 'datetime' by concatenating the 'date' and 'time' columns
sentiment_df['datetime'] = pd.to_datetime(sentiment_df['date'] + ' ' + sentiment_df['time'])

# Set the 'datetime' column as the index
sentiment_df = sentiment_df.set_index('datetime')

# Localize the times to UTC
sentiment_df = sentiment_df.tz_localize('utc')

# Display the first record of the datetime and inspect the index
display(sentiment_df.index[0])


In [ ]:
# Set the URL for the meaningcloud API
url = "https://api.meaningcloud.com/sentiment-2.1"

# Set the default payload for the API
payload={
    'key': os.getenv("MEANINGCLOUD_API_KEY"),
    'txt': '',
    'lang': 'en'
}

In [ ]:
# Set a counter to control the number of rows to process
runcount = 0

# Cycle through the rows of the dataframe and call API
for index, row in sentiment_df.iterrows():
    
    if runcount < 5000:
        
        # Increment the counter
        runcount = runcount + 1
        
        # Sleep for 1 second to avoid outrunning the meaningcloud rate limit
        time.sleep(1)
        
        # Print out the runcount and the Tweet text to be processed
        print(runcount)
        print(row['tweet'])

        # Clean the garbage characters out of the tweet text and set value in payload text
        payload['txt'] = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", row['tweet']).split())

        # Check to make sure the remaining text is not null
        if payload['txt'] is not None:
            # Print out the cleaned up tweet text
            print(payload['txt'])
            
            # Call the Meaningcloud API, passing the cleaned tweet text, getting response
            response = requests.post(url, data=payload).json()
            
            # Check to make sure the API returned a key called score_tag in the JSON response
            if "score_tag" in response:
                
                # Print out the score_tag and the confidence rating
                print(response['score_tag'])
                print(response['confidence'])
                
                # Update the dataframe with the new columns for score_tag and confidence
                sentiment_df.at[index,'score_tag'] = response['score_tag']
                sentiment_df.at[index,'confidence'] = response['confidence']         
    



In [ ]:
# Save the resulting dataframe to a CSV
# filepath = Path('sentiment_df.csv')
# sentiment_df.to_csv(filepath)

sentiment_df.to_csv('sentiment_df.csv')


## Processing of Yesterday (one day) Sentiment of Elon Tweets

In [2]:
## This cell has code to run yesterday's Elon tweet contents through the meaningcloud sentiment analyzer

# Import required libraries
import re
import requests
import pandas as pd
from textblob import TextBlob
import os
from dotenv import load_dotenv
import time

# Load env variables in from .env
load_dotenv()

# Set base URL for meaningcloud API
url = "https://api.meaningcloud.com/sentiment-2.1"

# Create payload to send to meaningcloud API
payload={
    'key': os.getenv("MEANINGCLOUD_API_KEY"),
    'txt': '',
    'lang': 'en'
}

# Read the CSV of yesterday's tweets and gather the sentiments from meaningcloud
yesterdays_elon_tweets_df = pd.read_csv('yesterday.csv')

# Iterate through yesterdays_tweets dataframe and get sentiment from meaningcloud API
for index, row in yesterdays_elon_tweets_df.iterrows():
    time.sleep(1)
    # Set the payload text to the raw tweet text
    payload['txt'] = row['Tweet']
    
    # Call meaningcloud API and store response as JSON
    response = requests.post(url, data=payload).json()
    
    # # Update the dataframe with the new columns for score_tag and confidence
    yesterdays_elon_tweets_df.at[index,'score_tag'] = response['score_tag']
    yesterdays_elon_tweets_df.at[index,'confidence'] = response['confidence']

# Create a column 'datetime' by concatenating the 'date' and 'time' columns
yesterdays_elon_tweets_df['datetime'] = pd.to_datetime(yesterdays_elon_tweets_df['date'] + ' ' + yesterdays_elon_tweets_df['time'])

# Set the 'datetime' column as the index
yesterdays_elon_tweets_df = yesterdays_elon_tweets_df.set_index('datetime')

# Localize the times to UTC
yesterdays_elon_tweets_df = yesterdays_elon_tweets_df.tz_localize('utc')

# Output the resulting dataframe to CSV
yesterdays_elon_tweets_df.to_csv('yesterdays_elon_tweets_df.csv')


